In [12]:
import cv2
import dlib
from scipy.spatial import distance as dist
import time
from pydub import AudioSegment
import os
import tempfile
import simpleaudio

In [13]:
# Specify the path to the FFmpeg executable
ffmpeg_path = "ffmpeg-6.0.tar.xz"  # Replace with the actual path to FFmpeg executable
# Set the environment variable for pydub
os.environ["PATH"] += os.pathsep + ffmpeg_path

In [14]:
# Load face detector and facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat') 

In [15]:
# Function to calculate eye aspect ratio (EAR)
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [16]:
# Set the threshold for drowsiness detection
EAR_THRESHOLD = 0.25  # Adjust this based on your observations
TIME_THRESHOLD = 1.5  # Time threshold for closed eyes (in seconds)

In [17]:
# Initialize timer variables
start_time = None
drowsy_timer = 0

In [18]:
# Load the alarm sound
alarm_sound = AudioSegment.from_wav("alarm.wav") 

In [19]:
# Specify a temporary directory where you have write access
temp_dir = "C:\\Users\\chait\\Downloads"

In [20]:
# Start capturing video from the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    for face in faces:
        landmarks = predictor(gray, face)
        left_eye = []
        right_eye = []
        
        for i in range(36, 42):  # Indices for left eye landmarks
            left_eye.append((landmarks.part(i).x, landmarks.part(i).y))
        for i in range(42, 48):  # Indices for right eye landmarks
            right_eye.append((landmarks.part(i).x, landmarks.part(i).y))
        
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        
        avg_ear = (left_ear + right_ear) / 2.0
        
        if avg_ear < EAR_THRESHOLD:
            if start_time is None:
                start_time = time.time()
            drowsy_timer = time.time() - start_time
            if drowsy_timer >= TIME_THRESHOLD:
                cv2.putText(frame, "Drowsy", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                temp_wav_filename = "alarm.wav"
                temp_wav_path = os.path.join(temp_dir, temp_wav_filename)
                alarm_sound.export(temp_wav_path, format="wav")
                wave_obj = simpleaudio.WaveObject.from_wave_file(temp_wav_path)
                play_obj = wave_obj.play()
                play_obj.wait_done()
        else:
            start_time = None
        
        # Draw landmarks and eyes on the frame (for visualization)
        for point in left_eye:
            cv2.circle(frame, point, 2, (0, 255, 0), -1)
        for point in right_eye:
            cv2.circle(frame, point, 2, (0, 255, 0), -1)
    
    cv2.imshow("Drowsiness Detection", frame)
    
    if cv2.waitKey(1) & 0xFF == 27:  
        break

cap.release()
cv2.destroyAllWindows()